Предполагается, что в рамках данного задания ученики сами будут изучать документацию, как минимум используя функционал `tab` и `shift+tab` от Jupyter

1. Импортируйте библиотеку pandas 
2. Считайте файл `wage.csv` в DataFrame с названием `wage`. Используйте для этого функцию pd.read_csv()

`person_id` - идентификатор человека

`gender` - пол, 0 - женский, 1 - мужской

`wage` - заработная плата в рублях

In [286]:
import pandas as pd
wage = pd.read_csv('e:wage.csv')

3. Поменяйте колонку `gender` так, чтобы были записи `F` (female) и `M` (male) вместо 0 и 1

In [287]:
wage.gender = wage.gender.apply(lambda x: 'F' if x == 0 else 'M')

4. Давайте посчитаем среднюю зарплату у мужчин и женщин. Для этого используйте метод `groupby` со следующим синтаксисом:

`dataframe.groupby(Название_колонки_для_группировки)[Перечисление_колонок_для_агрегации].функция_агрегации()`

In [288]:
wage\
    .groupby('gender', as_index = False)[['wage']]\
    .mean()\
    .rename(columns = {'wage' : 'mean_wage'})

,gender,mean_wage
0,F,40855.747261
1,M,46815.944005


5. Теперь взглянем внимательнее на данные и обнаружим, что некоторые люди попали в выборку несколько раз. 
    1. Найдите таких людей. Подсказка: `value_counts()`
    0. Убедитесь, что записи по ним с одинаковым `wage`. Возможно, тут вам пригодится функция агрегации `nunique()`, отображающая количество разных значений
    0. Избавьтесь от повторяющихся значений. Подсказка: `drop_duplicates()`

In [289]:
wage.value_counts()

person_id  gender  wage         
17         M       20178.985060     2
16         F       35448.040878     2
15         F       68341.332163     2
14         M       13434.655493     2
13         M       273483.609236    2
                                   ..
661        F       17181.940427     1
660        M       65520.795238     1
659        F       10461.600089     1
658        F       19783.977951     1
0          M       46793.603811     1
Length: 1000, dtype: int64

In [290]:
wage.person_id.nunique()

1000

In [291]:
wage.shape[0] == wage.value_counts().shape[0]

False

In [292]:
wage = wage.drop_duplicates()

In [293]:
wage.shape[0] == wage.value_counts().shape[0]

True

6. Теперь посмотрим внимательнее на зарплаты
    1. Охарактеризуйте имеющиеся данные по зарплатам. Подсказка: `describe`
    1. Избавьтесь от бессмысленных значений

In [294]:
wage[['wage']].describe()

,wage
count,1000.000000
mean,43694.227404
std,55352.539343
min,-287418.645743
25%,14489.682367
50%,27309.529498
75%,52021.080258
max,755320.874132


In [295]:
wage = wage[wage.wage > 0]

In [296]:
wage[['wage']].describe()

,wage
count,995.000000
mean,44306.969585
std,54302.194392
min,945.648458
25%,14683.306148
50%,27519.361794
75%,52267.313664
max,755320.874132


7. Давайте теперь посмотрим на зарплату с учетом бонуса. Для этого нам понадобится таблица `bonus.csv`. Считайте ее в переменную `bonus`. Заметьте, что она сохранена немного в другом формате, и вам понадобится уточнить параметр `sep` - разделитель записей. Сравните текущий файл с предыдущим и попробуйте решить проблему

In [297]:
bonus = pd.read_csv('e:bonus.csv', sep = ';')

8. Чтобы посчитать итоговую зарплату, нам нужно по каждому человеку знать и оклад, и премию. Для этого надо будет соединить (сджойнить) таблицы по `person_id`. Используйте для этого функцию `pd.merge`. Помните, что параметр `how` должен быть `'outer'`, чтобы сохранить те записи, что есть только в одной таблице. Результат запишите в новый dataframe `df`

In [298]:
df = wage.merge(bonus, how = 'outer', on = 'person_id')

9. Наконец, давайте посчитаем итоговую зарплату
    1. Замените отсутствующие записи в колонке `bonus` нулями
    1. Уберите людей без `wage` - это те "плохие" записи, от которых мы избавлялись на предыдущих шагах
    1. Сделайте новую колонку `total`, которая будет равна 12 окладам и премии
    1. Посчитайте среднюю и медианную итоговую зарплату в разрезе по полу. Подсказка: вместо функции агрегации можно написать `.agg()` и перечислить внутри нужные агрегаты

In [299]:
df = df.fillna(0)

In [300]:
df = df[df.wage > 0].copy()

In [302]:
df['total'] = df.wage * 12 + df.bonus

In [316]:
df.groupby('gender').agg({'total' : 'mean'})

,total
gender,
F,570746.139432
M,657142.490282


In [317]:
df.groupby('gender').agg({'total': 'median'})

,total
gender,
F,347622.913892
M,437499.824868


10. Сохраните `df` в файл, используя метод `to_csv()`. Не записывайте индексы

In [319]:
df.to_csv('total_wage', index=False)